# crear clase para abrir archivo de configuración

In [ ]:
import numpy as np
from configparser import ConfigParser
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import *
# la clase carga los datos que se obtiene de la calibración.
#path direccion absoluta del lugar donde se encuentra el archivo de configuración
#nombre del archivo de configuración
class configuracion:
    def __init__(self,path,nombre_config,area=True):
        self.path=path
    #secciones
        self.secciones=['homografia', 'calibracion_uv_depth', 'calibracion_uv_rgb','calibracion_z']
        #caracteristicas rgb
        self.rgb_mtx=0
        self.rgb_intrinseca=0
        self.rgb_coef_dist_=0
        self.rgb_roi=0
        self.rgb_inversa=0
        #caracteristicas profundidad
        self.depth_mtx=0
        self.depth_intrinseca=0
        self.dpth_coef_dist_=0
        self.depth_roi=0
        self.depht_inversa=0
        #cambio de dominio
        self.homografia=0
        #calibracion z
        self.pendientes=0
        self.interceptos=0
        #ejecución de funciones
        #cargar archivo de configuracion
        self.config = ConfigParser()
        #revisar existencia del archivo  y comprobar existencia de todas las secciones
        ret1,ret2=self.comprobar(nombre_config)
        if not(ret1):
            print("el achivo no existe ")
        if not ret2:
            print("hay un numeor invalido de secciones")
        #cargar valores
        self.rgb_mtx,self.rgb_intrinseca,self.rgb_inversa,self.rgb_coef_dist,self.rgb_roi=self.calibracion_uv("rgb")
        self.depth_mtx,self.depth_intrinseca,self.depth_inversa,self.depth_coef_dist,self.depth_roi,=self.calibracion_uv("depth")
        self.homografia=self.calibracion_homografia()
        self.pendientes,self.interceptos=self.calibracion_z()
        if area: # debido a compatibilidad
            self.polinomio=self.calibracion_area()
        else:
            self.polinomio=0
    # comprobar si todas las secciones se encuentran
    def comprobar(self,nombre):
     #revisar existencia del archivo
        ret1=self.config.read('{}{}.ini'.format(self.path,nombre))
    # comprobar que se encuentren todas las secciones
        secciones=self.config.sections()
        ret2=True
        for i in range(len(self.secciones)):
            ret2=ret2 and(self.secciones[i]in secciones)
        return ret1, ret2
    def calibracion_uv(self,tipo):
        #hay que convertir el texto a matriz 
        datos=(self.config["calibracion_uv_{}".format(tipo)])
        mtx=self.preparar_dato(datos["mtx"],3,3)
        mt_intrinseca=self.preparar_dato(datos["mt_intrinseca"],3,3)
        mt_inversa=self.preparar_dato(datos["mt_inversa"],3,3)
        roi=self.preparar_dato(datos["roi"],1,4)
        coef_dist=self.preparar_dato(datos["coef_dist"],1,5)
        return mtx,mt_intrinseca,mt_inversa,coef_dist,roi
    def calibracion_homografia(self):
        datos=self.config["homografia"]
        homografia=self.preparar_dato(datos["homografia"],3,3)
        return homografia
    def calibracion_z(self):
        datos=self.config["calibracion_z"]
        shape=(self.preparar_dato(datos["shape"],1,2).reshape(-1)).astype(np.int)
        margen_v=int(datos["margen_v"])
        margen_h=int(datos["margen_h"])
        pendientes_marco=[]
        interceptos_marco=[]
        alto=shape[0]-2*margen_v
        ancho=shape[1]-2*margen_h
        for i in range(alto*ancho):
            pendientes_marco.append(float(datos["pendiente{}".format(i)]))
            interceptos_marco.append(float(datos["intercepto{}".format(i)]))
        pendientes_marco=np.array(pendientes_marco)
        interceptos_marco=np.array(interceptos_marco)
        # colocar mapa en dimensiones de la imagen
        pendientes=np.ones(shape)
        interceptos=np.zeros(shape)
        pendientes[margen_v:-margen_v,margen_h:-margen_h]=pendientes_marco.reshape(alto,ancho)
        interceptos[margen_v:-margen_v,margen_h:-margen_h]=interceptos_marco.reshape(alto,ancho)
        
        return pendientes,interceptos
    def calibracion_area(self):
        datos=self.config["area"]
        polinomio=self.preparar_dato(datos["polinomio"],-1,1).reshape(-1)
        return polinomio
    #dato:string que contine un vector fila en formato texto. el string tambien contine las llaves que indican que es un vector fila
    # a pesar de ser un estring, cada elemento se encuentra separado por un espacio.
    #nf:para convertir la entrada a una matriz. núero de filas
    #nc:para converitr la entrada a una matriz. número de columnas
    #nf
    def preparar_dato(self,dato,nf,nc):
        dato=dato.lstrip("[")#quitar llave lado izquierdo
        dato=dato.rstrip("]")#quitar corchete lado derecho
        dato=dato.split()# separa el string teniendo en cuenta los espacios, regresa  una lista de estring.
        dato=np.array(dato,dtype=np.float).reshape(nf,nc)#convirte la lista a un array numpy. cada elemento se convierte a float64. luego se convierte en una matriz
        return dato
        
    

# prueba clase configuración

In [ ]:
#from medicion import *
a=configuracion("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_120/","configuracion_d5_90 (d5)",area=True)
print(a.polinomio)
print(a.depth_mtx)
print(a.interceptos.shape)

# crear clase imagen
1. la clase resive una imagen rgb o de profundidad y le quita la distorcion causada por los lentes de la camara.
2. la clase tiene una función que resive una imagen rgb y la lleva al dominio de profundidad.


In [ ]:
import cv2
import numpy as np
import os
from medicion import configuracion
#el contructor de la clase resive el nombre y direccion absolta del lugar donde se encuentra el archivo de configuracion que contiene los parametros de calibración
# esta clase tiene funciones que resiven una imagen y le quitan la distorcion ya sea para el sensor rgb o el sensor de profundidad
#esta clase tambien tiene una función que lleva una imagen del dominio rgb al dominio de profundidad
# la clase posee una funcion para poder abrir un par de imagenes que reprensenten la misma escena
class imagen(configuracion):
    def __init__(self,path,nombre_config):
        configuracion.__init__(self,path,nombre_config)
    # quitar distorsion causado por lentes a la imagen rgb
    def distorsion_rgb(self,img):

        dst = cv2.undistort(img, self.rgb_mtx, self.rgb_coef_dist, None, self.rgb_intrinseca)

        return dst
    # quitar distorsion causado por lentes a la imagen de profundidad
    def distorsion_depth(self,img):
     
        dst = cv2.undistort(img.copy(), self.depth_mtx, self.depth_coef_dist, None, self.depth_intrinseca)
        
        return dst
    #quitar distorsión causada por lentes a par de imagenes
    def distorsion(self,img_rgb,img_depth):
        dst_rgb = cv2.undistort(img_rgb, self.rgb_mtx, self.rgb_coef_dist, None, self.rgb_intrinseca)
        dst_depth = cv2.undistort(img_depth, self.depth_mtx, self.depth_coef_dist, None, self.depth_intrinseca)
        return dst_rgb,dst_depth
    def distorsion_z(self,img_depth,filtrar):
        dst_depth=img_depth.copy()
        if filtrar:
            dst_depth=self.filtrar_depth(dst_depth,blur=3,radio=40,sigma_espacio=80,sigma_color=50)
        dst_depth=dst_depth*self.pendientes+self.interceptos
        return dst_depth.astype(np.float32)
    # pasar una imagen del dominio rgb al dominio de profundidad
    def rgb2_depth(self,img):
        img_depth= cv2.warpPerspective(img.copy(), self.homografia, (img.shape[1], img.shape[0]))#primero ancho y luego alto
        return img_depth
    def filtrar_depth(self,img_depth,blur,radio,sigma_espacio,sigma_color):
        img=img_depth.copy().astype(np.float32)
        img=cv2.blur(img,(blur,blur))
        img = cv2.bilateralFilter(img,radio,sigma_espacio, sigma_color)
        return img
    
    # abre el par de imagenes rgb y profundidad que esten captando la misma escena
    #los nombre desben estar en el formato indicador_nombre. indicardor="rgb" para la imagen del sensor rgb
    #indicador="depth" para la imagen del sensor de profundidad. 
    #nobre debera ser el mismo para el par de imagenes que representen la misma escena.
    def abrir_par_img(self,path,indicador_rgb,indicador_depth,nombre):
        path_rgb=os.path.join(path, '{}_{}.png'.format(indicador_rgb,nombre))
        path_depth=os.path.join(path, '{}_{}.png'.format(indicador_depth,nombre))
        #confirmar existencia de ambas imagenes
        ret_2=cv2.haveImageReader(path_depth)
        ret_1=cv2.haveImageReader(path_rgb)
        img_rgb=0;img_depth=0;
        ret=ret_1 and ret_2#
        if ret:
            img_depth=cv2.imread(path_depth,cv2.IMREAD_UNCHANGED)[:,:,0]
            img_rgb=cv2.imread(path_rgb)
           
        return ret,img_rgb,img_depth
    def preprocesar_imgs(self,img_rgb,img_depth,f_segmentador):
        #quitar distorsion z
        img_depth=self.distorsion_z(img_depth,filtrar=True)
     
        #quitar distorsion uv
        img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        # cambiar de dominio rgb a depth
        img2_depth=self.rgb2_depth(img_rgb)
        # encontrar mascara
        mascara=f_segmentador(img2_depth)
        return img_depth,mascara
    
    

# prueba clase

In [ ]:
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import graficar

preparar=imagen("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion/","configuracion_union")
#abrir imagen
path="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/longitud"
ret,img_rgb,img_depth=preparar.abrir_par_img(path,"gray","depth",104)
# quitar distorsion
print(ret)
img_rgb2,img_depth2=preparar.distorsion(img_rgb,img_depth)
#quitar distorsion en z
img_depth3=preparar.distorsion_z(img_depth2,1)
#llevar rgb al dominio de profundidad
img2_depth=preparar.rgb2_depth(img_rgb)

graficar([img_rgb,img_rgb2,img_depth,img_depth2,img2_depth,img_depth3],3,2,["imagen rgb original","imagen rgb sin distorsión","imagen depth original","imagen depth sin distorsión","imagen rgb en dominio de profundidad","imagen profundidad sin distorsión en z"])


    

In [ ]:
print(img_depth3.dtype)
print(img_depth2.dtype)

# crear clase longitud
1. la clase realizara medicion de distancias entre los puntos del patron
2. a partir de los puntos centro del patron se calculara  los puntos medios entre circulos de una misma fila.
3. una imagen rgb se distorciona y luego se lleva al dominio de profundidad.
     1. a la imagen que se le hizo cambio de cominio se le hayan los puntos, obteniendo cordenadas uv en profundidad.
4. se forma los puntos (u,v,z) con la informacion de la imagen de profundidad.
    1. se convirerte a cordenadas (x,y,z)
    2. se haya distancia entre los puntos de cordenadas (x,y,z)
    3. se calcula el error con respecto al patron

nota: se debe calcular una nueva grilla debido a en la imagen de profundidad en el punto centro es probable que no haya informacion de profundidad.


In [ ]:
from medicion import *
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import *
# la clase podra calcular la distancia entre los puntos medio de los circulos por fila del patron.
#la  tendra una función que resive la informacion de una escena. 2 imagenes. imagen rgb e imagen de profundidad que represente la misma escena
#a la imagen rgb se le quita la distorsion se la lleva al dominio de profundidad y alli se encuentran los puntos uv, puntos imange.
# se calcula los puntos (x,y,z) a travez de los puntos uv y la informacion de la camara de profundidad.
# se encuentra la distancia entre los puntos (X,Y,Z) y se calcula el error con respecto a las medidas del patron
# en el constructor se inicializa las variables de la calibración y la iformacion util para extraer los puntos.

class longitud(imagen,puntos):
    # path_configuracion: dirección absoluta del lugar donde se encuentra el archivo de configuración
    #nombre_configuración: nombre del archivo configuración sin la extensión .ini
    #d:diametro de circulos en milimetros
    # nf: numero de filas del patron
    #nc: numero de circulos por fila del patron
    def __init__(self,path_configuracion,nombre_configuracion,d,nf,nc):
        imagen.__init__(self,path_configuracion,nombre_configuracion)
        puntos.__init__(self,nf,nc)
        # variables de isntancia
        self.distancia_patron=self.formar_distancia(d)# distancia original entre puntos contiguos del patron. tiene en cuenta que los puntos estan agrupados por fila, los puntos de cada fila estan ordenados de izquierda a derecha, y las filas de arriba hacia abajo.
        self.error_promedio=0
    # abre el par de imagenes rgb y profundidad que esten captando la misma escena
    #los nombre desben estar en el formato indicador_nombre. indicardor="rgb" para la imagen del sensor rgb
    #indicador="depth" para la imagen del sensor de profundidad. 
    #nobre debera ser el mismo para el par de imagenes que representen la misma escena.
    #pasa la imagen rgb al dominio de profundidad
    #
    def obtener_uvz(self,path, indicador1,indicador2,nombre):
        #abrir imagenes
        ret,img_rgb,img_depth=self.abrir_par_img(path,indicador1,indicador2,nombre)
        print(ret)
        #quitar distorision z
        #
        #quitar distorsion
        img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        img_depth=self.distorsion_z(img_depth,filtrar=True)
        # cambiar de dominio
        img2_depth=self.rgb2_depth(img_rgb)
        # hallar puntos medios entre los circulos de una misma fila
        ret,_,puntos_uv=self.extraer(img2_depth,0)# es una imagen en el dominio de profundidad pero su caracteristicas siguen siendo de color. no necesita mejora
        # informar si hubo detección de puntos
        print("hubo de detección de puntos?: {}".format(ret))
        #obtener cordenada z # numpy indexa fila x columna, opencv indexa ancho por lato. por lo tanto hay que invertir los indices.
        z=[]
        if ret:
            
            for punto in puntos_uv:
                fila=int(np.around(punto[0][1]))
                col=int(np.around(punto[0][0]))
                valor=img_depth[fila,col]

                z.append(valor)
            z=np.array([z])
        #dibujar puntos medios sobre la imagen de profundidad
        img_puntos=self.dibujar(img_depth,puntos_uv,1)
        return ret,puntos_uv,z,img_puntos
    # la función resive puntos uv y los trasforma a puntos X,Y,Z
    # reproyecta los puntos uv encontrados en la imagen y los lleva al espacio.las cordenadas son cartesians y su origen  es el foco de la camara
    def obtener_xyz(self,puntos_uv,z):
        
        # los puntos estan almacenados en vectores fila [u,v]. para la multiplicacion matricial se los lleva a vectores tipo columna [u;v;1]
        puntos_uv=puntos_uv.reshape(-1,2) # desepaquetar. de shape(n,1,2) hacia shape(n,2)
        puntos_u=puntos_uv[:,0]# extraer todas las componentes u de los puntos
        puntos_v=puntos_uv[:,1]# #extraer todas las componentes v
        unos=np.ones(puntos_uv.shape[0])
        puntos_uv1=np.array((puntos_u,puntos_v,unos))# formar vectores columna (u;v;1)
        puntos_xy1_n=np.dot(self.depth_inversa,puntos_uv1)# calcular cordenadas normalizadas
        puntos_xyz=z*puntos_xy1_n# calular cordenadas espaciales de la forma [X;Y;Z]. son vectores columna
        #devolver puntos a forma vector fila [(x,y,z)].separando los putos
        puntos_xyz=np.concatenate((puntos_xyz[0].reshape(-1,1),puntos_xyz[1].reshape(-1,1),puntos_xyz[2].reshape(-1,1)),axis=1)
        return (puntos_xyz) #entregar array de la forma shape(n,3)
    #encuentra la distancia euclidiana entre los puntos contiguos XYZ y los compara  con la distancia de construcción de los puntos del patron
    def calcular_distancia(self,puntos_xyz):
        distancias=[]
        error_distancias=[]
        for i in range(puntos_xyz.shape[0]-1):
            vector=puntos_xyz[i+1]-puntos_xyz[i]
            distancia=np.sqrt(np.sum(np.power(vector,2)))
            distancias.append(distancia)
        distancias=np.array(distancias).reshape(-1,1)/10
        error_distancias=np.absolute(self.distancia_patron-distancias)
        error_distancias=100*error_distancias/self.distancia_patron
        self.error_promedio=np.sum(error_distancias)/error_distancias.size
        return distancias,error_distancias,self.error_promedio
    #la distancia se calcula entre dos puntos contiguos
    #d=diametro de los circulos
    #nf=numero de filas del patron
    #nc=numero de circulos por fila
    #los puntos estan agrupados por filas, los puntos de cada fila se encuentran ordenados de izquierda a derecha y las filas de arriba hacia abajo
    def formar_distancia(self,d):
        nc=self.nc-1# al ser puntos medio entre circulos se pierde un punto por cada fila
        distancias_patron=np.ones((self.nf*nc-1,1))#al ser distancia sobre puntos contiguos la informacion disminuye en un dato
        #formar distancias teniendo en cuenta la estructura del patron
        for i in range(distancias_patron.size):
            if (i+1)%nc:# puntos contiguos de una misma fila
                distancias_patron[i]=2*d
            else:#puntos contiguos, punto extremo derecho fila superior, punto extremo izquierdo fila inferior
                if (i+1)%10:# por se patron asimetrico,las distancias entre extremos cambian de una fila a otra
                    distancias_patron[i]=np.sqrt((2*(nc-1)*d-d)**2+d**2)#fila impar
                else:
                    distancias_patron[i]=np.sqrt((2*(nc-1)*d+d)**2+d**2)#fila par
        return distancias_patron

    
        
    

In [ ]:
#from medicion import *
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import *

medir=longitud("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion/","configuracion_union",50,9,6)
ret,puntos_uv,z,img_puntos=medir.obtener_uvz("/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/longitud","gray","depth",56)

if ret:
        print("distancia media",z.mean())
        xyz=medir.obtener_xyz(puntos_uv,z)
        distancias,error_distancias,error_promedio=medir.calcular_distancia(xyz)
        for i in range(distancias.size):
            print("puntos {} y {} : las siguientes son unidades en milimetros".format(i,i+1))
            print("distancia calculada:{}".format(distancias[i]))
            print("distancia original:{}".format(medir.distancia_patron[i]))
            print("error en porcentaje :{}%".format(error_distancias[i]))
        print("error promedio en porcentaje :{}%".format(error_promedio))
        graficar([img_puntos],1,1,["imagen depth con puntos detectados"])






In [ ]:
from tabulate import tabulate
tabla=[]
for distancia,distancia_patron,error,i in zip(distancias,medir.distancia_patron,error_distancias,range(distancias.size)):
    tabla.append(["{} y {}".format(i,i+1),distancia,distancia_patron,error])
print("error promedio",error_distancias.mean())
print(tabulate(tabla,headers=["puntos","V medido[mm]","V Real[mm]","error%"]))
    


# analisis
0. la profundidad media en la que se calcularon la longitudes es alrededor de 1m.
1. cuando la matriz homográfica se calcula sin quitar distorision y tambien y tampoco se le quita a las imagenes a las cuales se les mide las longitudes, el eror promedio es del 6.06%
2. cuando la matriz homografica se calcula sin distorsión pero si se le quita a las imagenes a las cuales se les mide las longitudes, el error promedio es de 2.43
3. cuando se calcula la matriz homografica teniendo en cuenta la distorsion y tambien se quita la distorsión a las imagnes, el error es de de 1.22
4. quitar distorsion z,sin filtrar y  antes de quitar la distorsion uv baja el error promedio de 1.217 a 1.213.
5. quitar la distorsion z filtrando y antes de quitar la distorsion uv baja el error  promedio de 1.217 a 1.194
6. Se encontro una mejora aplicando distorsion z, antes que distorsion uv. Cuando se realiza distorsion uv antes que Z, aumento de 1.27 a 1.346 sin filtrar, y 1.319 con filtro.  


# segmentado objeto negro fondo blanco
1. la funcion resive una imagen RGB y devuelve una imagen binaria. pixeles en uno pertenecen al objeto. pixeles en cero pertenecen al fondo.


In [ ]:

from skimage import morphology
import numpy as np
import cv2
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import graficar
def segmetador(img,negro=True):
    n=200
    kerner=kernel = np.ones((n,n),np.uint8)
    img_gray=cv2.cvtColor(img.copy(), cv2.COLOR_RGB2GRAY)
    #img_= cv2.medianBlur(img_gray,15)
    if negro:
        criterio= cv2.THRESH_BINARY+cv2.THRESH_OTSU
        img_gray = cv2.morphologyEx(img_gray, cv2.MORPH_BLACKHAT, kernel)#MORPH_BLACKHAT
       
    else:
        criterio=cv2.THRESH_BINARY+cv2.THRESH_OTSU
    thr,mascara =cv2.threshold(img_gray,0,1,criterio)
    mascara.dtype=bool## remove_small solo funciona con variables binarias
    mascara=morphology.remove_small_objects(mascara,50,connectivity=10)
    mascara.dtype=np.uint8
    img_mascara=cv2.bitwise_and(img,img,mask=mascara)
    return thr,mascara,img_mascara

# abrir img
img=cv2.imread("/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/area/gray_2.png")
thr,mascara,img_mascara=segmetador(img,1)
print("humbral: {}".format(thr))

graficar([img,mascara,img_mascara],1,3,["original","mascara","original X mascara"])


# medición de area en cordenadas uv
1. la clase debe cargar el archivo de configuracin que contiene los datos de la calibración
2. la case debe funcionar independiente del segmentador
3. la clase tendra una función que resive un par de imagenes. RGB y profundidad y le quita la distorsión 
4. la clase tendra una función que resiva una imagen de profundidad y una mascara y apartir de ello calculara area.
5. la clase tendra una función que realizara el cambio de dominio de RGB al dominio de profundidad antes de  entrar la imagen a un segmentador.


In [ ]:
from medicion import *
from tabulate import tabulate
class area(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion)
        self.fx=self.depth_intrinseca[0,0]
        self.fy=self.depth_intrinseca[1,1]
        self.constante_area=1/(self.fx*self.fy)
    def abrir_imgs(self,path, indicador1,indicador2,nombre):
        #abrir imagenes
        img2_depth=0
        img_depth=0
        ret,img_rgb,img_depth=self.abrir_par_img(path,indicador1,indicador2,nombre)
       
        if ret:
            #quitar distorsion z
            img_depth=self.distorsion_z(img_depth,filtrar=True)
            #quitar distorsion
            img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
            # cambiar de dominio
            img2_depth=self.rgb2_depth(img_rgb)
        return ret,img2_depth,img_depth
    def calcular_area(self,img,mascara):
        img_z=cv2.bitwise_and(img,img,mask=mascara)
        img_z=np.array(img_z/10,dtype=np.float64)
        #print("distancia maxima de profundidad en mm: {}".format(img_z.max()))
        img_z2=img_z*img_z
        img_area=self.constante_area*img_z2
        area=np.sum(img_area)
        return area
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e

# Area Circulos

In [ ]:
from medicion import *
from tabulate import tabulate
list_tablas=[]
list_config=["configuracion_d5_70",
            "configuracion_d5_90",
            "configuracion_d5_110",
            "configuracion_d5_130",
#             "configuracion_d5_150",
#             "configuracion_d5_170",
            "configuracion_d10_120",
            "configuracion_d10_130",
            "configuracion_d10_140",
            "configuracion_d10_150",
            "configuracion_d10_170",
            "configuracion_distorsion_d5_70",
            "configuracion_distorsion_d5_90",
            "configuracion_distorsion_d5_110",
            "configuracion_distorsion_d5_130",
            #"configuracion_distorsion_d5_150",
            #"configuracion_distorsion_d5_170",
            "configuracion_distorsion_d10_120",
            "configuracion_distorsion_d10_130",
            "configuracion_distorsion_d10_140",
            "configuracion_distorsion_d10_150",
            "configuracion_distorsion_d10_170"]
list_config=["configuracion_d5_90"]
ejex=[]
ejey=[]
list_titulos=["error porcentual"]
for config in list_config:
    print(config)
    a=area("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_normal/",config)
        # abrir el par de imagenes
    list_medidas=[]
    list_error=[]
    list_media=[]
    for i in range(1,100):
        ret,img2_depth,img_depth=a.abrir_imgs(path="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/circulos",
                                        indicador1="gray",
                                        indicador2="depth",
                                        nombre=i
                                       )
        if ret:
            #segmentar
            thr,mascara,_=segmetador(img2_depth)
            #visualizar 
            #graficar([img_depth,img2_depth,mascara],1,3,["img original profundidad","img rgb en profundidad","mascara"])

            area_calculada=a.calcular_area(img_depth,mascara)
            valor_real=((150.0**2)*np.pi)/4
            #valor_real=10000
            error=a.calcular_error(valor_real,area_calculada)
            list_medidas.append(area_calculada)
            list_error.append(error)
            list_media.append(img_depth.mean()/10)
#             print("valor medio profundidad mm: ",img_depth.mean()/10)
#             print("valor real mm²: {}".format(valor_real))
#             print("area calculada mm²:{}".format(area_calculada))
#             print("porcentaje de error:{}".format(error))
#     print(valor_real)
    tabla=zip(list_medidas,list_error,list_media)
    ejex.append(list_media)
    ejey.append(np.absolute(list_error))
    list_titulos.append(config)
    print("valor real:{}".format(valor_real))
    print(tabulate(tabla,headers=["v calculado","error","distancia media"]))
    plot([list_media],[np.absolute(list_error)],1,1,["Area círculos"],[["Profundidad","Error absoluto %"]],tipo=1)
    list_media_circulos=list_media
    circulos=list_error


In [ ]:
print(len(list_tablas))

# analisis area 
1. Si no se quita distorsión en el calculo de matriz homografica H para el cambio de dominio ni tampoco al abrir la imagen en el mejor de los casos el error promedio es 9.99
2. Si no se quita distorsion al abrir la imagen, pero se calcula H teniendo en cuenta la distorsion, el error promedio es 5.24
3. Si se tiene en cuenta la distorsion de ambos casos 4

# comparacion area circulos con y  distorsión en z


In [ ]:
from medicion import *
class area_z(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion)
        self.fx=self.depth_intrinseca[0,0]
        self.fy=self.depth_intrinseca[1,1]
        self.constante_area=1/(self.fx*self.fy)
    def abrir_imgs(self,path, indicador1,indicador2,nombre,distorsion_z=-1):
        #abrir imagenes
        ret,img_rgb,img_depth=self.abrir_par_img(path,indicador1,indicador2,nombre)
        img2_depth=0
        if ret:
            if distorsion_z!=-1:
            #quitar distorsion z
                img_depth=self.distorsion_z(img_depth,filtrar=distorsion_z)
                #quitar distorsion
            img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
            # cambiar de dominio
            img2_depth=self.rgb2_depth(img_rgb)
        return ret,img2_depth,img_depth
    def calcular_area(self,img,mascara):
        img_z=cv2.bitwise_and(img,img,mask=mascara)
        img_z=np.array(img_z/10,dtype=np.float64)
        #print("distancia maxima de profundidad en mm: {}".format(img_z.max()))
        img_z2=img_z*img_z
        img_area=self.constante_area*img_z2
        area=np.sum(img_area)
        return area
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e

In [ ]:
from medicion import *
from tabulate import tabulate
def comparar_area(distorsion_z):
    a=area_z("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion/","configuracion_d5_110")
        # abrir el par de imagenes
    list_medidas=[]
    list_media=[]
    for i in range(1,100):
        ret,img2_depth,img_depth=a.abrir_imgs(path="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/piñon/",
                                        indicador1="gray",
                                        indicador2="depth",
                                        nombre=i,
                                        distorsion_z=distorsion_z
                                       )
        if ret:
            #segmentar
            thr,mascara,_=segmetador(img2_depth)
            area_calculada=a.calcular_area(img_depth,mascara)
            #valor_real=10000
            #valor_real=((150.0**2)*np.pi)/4
            #valor_real=14614.18
            valor_real=15524.847
            error=a.calcular_error(valor_real,area_calculada)
            list_medidas.append([area_calculada,error])
            list_media.append(img_depth.mean()/10)
    list_medidas=np.array(list_medidas)
    return list_medidas[:,0],list_medidas[:,1],np.array(list_media)
#area con distorison en z
medida0,error0,media=comparar_area(distorsion_z=-1)
#area sin distorision en z
medida1,error1,_=comparar_area(distorsion_z=0)
#area sin distorsion en z y con filtro
medida2,error2,_=comparar_area(distorsion_z=1)
tabla_error=zip(error0,error1,error2,media)
tabla_medidas=zip(medida0,medida1,medida2,media)
print(tabulate(tabla_error,headers=["con distorsión","sin distorsión","con filtro","distancia media"]))

      

# analis comparacion con y sin distorsion en z
1. como se mostro en la medida de longitud, en area ocurre lo mismo hay una reducción del error cuando se quita la distorsión en z antes de quitar distorsion en uv. realizar el filtro tambien representa una mejora,. pero no es significativa.
2. en longigud bajod de 1.217% a 1.194% a una profundidad media 97 cm, en area bajo de 5,66% a 3.80% a una profundidad media de 107 cm
3. se decide implemetar la distorsion en z con un filtro. la distorsion se presenta como una mejora reducinedo el error en 1.2 porcentuales unidades. un fitro espacial sin duda beneficiara objetos con cambios suaves en su forma.

# area poligono

In [ ]:
from tabulate import tabulate
from medicion import *
a=area("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_normal/","configuracion_d5_90")
    # abrir el par de imagenes
list_medidas=[]
list_error=[]
list_media=[]
for i in range(1,100):
    ret,img2_depth,img_depth=a.abrir_imgs(path="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/poligono",
                                    indicador1="gray",
                                    indicador2="depth",
                                    nombre=i
                                   )
    if ret:
        #segmentar
        thr,mascara,_=segmetador(img2_depth)
        #visualizar 
        graficar([img_depth,img2_depth,mascara],1,3,["img original profundidad","img rgb en profundidad","mascara"])

        area_calculada=a.calcular_area(img_depth,mascara)
        #valor_real=10000
        #valor_real=((150.0**2)*np.pi)/4
        valor_real=14614.18
        #valor_real=15524.847
        error=a.calcular_error(valor_real,area_calculada)
        list_medidas.append(area_calculada)
        list_error.append(error)
        list_media.append(img_depth.mean()/10)
        print("valor medio profundidad mm: ",img_depth.mean()/10)
        print("valor real mm²: {}".format(valor_real))
        print("area calculada mm²:{}".format(area_calculada))
        print("porcentaje de error:{}".format(error))
print(valor_real)
tabla=zip(list_medidas,list_error,list_media)
print("valor real:{}".format(valor_real))
print(tabulate(tabla,headers=["v calculado","error","distancia media"]))
plot([list_media],[np.absolute(list_error)],1,1,["Area círculos"],[["Profundidad","Error absoluto %"]],tipo=1)
list_media_poligono=list_media
poligono=list_error

# area piñon

In [ ]:
#from medicion import *
a=area("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_normal/","configuracion_d5_90")
    # abrir el par de imagenes
list_medidas=[]
list_error=[]
list_media=[]
for i in range(1,100):
    ret,img2_depth,img_depth=a.abrir_imgs(path="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/piñon/",
                                    indicador1="gray",
                                    indicador2="depth",
                                    nombre=i
                                   )
    if ret:
        #segmentar
        thr,mascara,_=segmetador(img2_depth)
        #visualizar 
        #graficar([img_depth,img2_depth,mascara],1,3,["img original profundidad","img rgb en profundidad","mascara"])

        area_calculada=a.calcular_area(img_depth,mascara)
        #valor_real=10000
        #valor_real=((150.0**2)*np.pi)/4
        #valor_real=14614.18
        valor_real=15524.847
        error=a.calcular_error(valor_real,area_calculada)
        list_medidas.append(area_calculada)
        list_error.append(error)
        list_media.append(img_depth.mean()/10)
        print("valor medio profundidad mm: ",img_depth.mean()/10)
        print("valor real mm²: {}".format(valor_real))
        print("area calculada mm²:{}".format(area_calculada))
        print("porcentaje de error:{}".format(error))
print(valor_real)
tabla=zip(list_medidas,list_error,list_media)
print("valor real:{}".format(valor_real))
print(tabulate(tabla,headers=["v calculado","error","distancia media"]))
plot([list_media],[np.absolute(list_error)],1,1,["Area Piñó"],[["Profundidad","Error absoluto %"]],tipo=1)
list_media_piñon=list_media
piñon=list_error

# area cartulina


In [ ]:
a=area("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/","configuracion_d5_50")
    # abrir el par de imagenes
for i in range(1,15):
    ret,img2_depth,img_depth=a.abrir_imgs(path="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/area2/",
                                    indicador1="gray",
                                    indicador2="depth",
                                    nombre=i
                                   )
    #segmentar
    thr,mascara,_=segmetador(img2_depth)
    #visualizar 
    graficar([img_depth,img2_depth,mascara],1,3,["img original profundidad","img rgb en profundidad","mascara"])

    area_calculada=a.calcular_area(img_depth,mascara)
    valor_real=322284
    error=a.calcular_error(valor_real,area_calculada)
    print("valor real mm²: {}".format(valor_real))
    print("area calculada mm²:{}".format(area_calculada))
    print("porcentaje de error:{}".format(error))

# resumen datos de calculo de area

In [ ]:
plot([list_media_circulos,list_media_poligono,list_media_piñon],[np.absolute(circulos),np.absolute(poligono),np.absolute(piñon)],list_titulos=[" Error de area","Circulo","Polígono","piñón"],ejes=[["Profunidad mm","error absoluto %"]],tipo=1)

# medición de area en el espacio

In [ ]:
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import *
import numpy as np

class area(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
        imagen.__init__(self,path_configuracion,nombre_configuracion)
        self.fx=self.depth_intrinseca[0,0]
        self.fy=self.depth_intrinseca[1,1]
        self.cx=self.depth_intrinseca[0,2]
        self.cy=self.depth_intrinseca[0,2]
        self.xn=0
        self.yn=0
        self.fila_expancion=0
        self.colum_expancion=0
    def calcular_area(self,img,mascara):
        #crear mapa de cordenadas uv y pasar a cordenadas normalizadas.
        if (isinstance(self.yn*self.xn,int)):
            v,u=np.mgrid[0:img.shape[0],0:img.shape[1]]
            self.yn=(v-self.cy)/self.fy
            self.xn=(u-self.cx)/self.fx
            self.fila_expancion=np.zeros([img.shape[0],1])
            self.colum_expancion=np.zeros([1,img.shape[1]])
        
        img_z=np.array(img/10,dtype=np.float64)
        img_x=self.xn*img_z
        img_y=self.yn*img_z
#         img_z=cv2.bitwise_and(img,img,mask=mascara)
       
        # generar distancias en x y en y
       
        img_dx=np.hstack([self.fila_expancion,img_x])-np.hstack([img_x,self.fila_expancion])
        img_dx=np.hstack([img_x,self.fila_expancion])-np.hstack([self.fila_expancion,img_x,])
       
        img_dy=np.vstack([img_y,self.colum_expancion])-np.vstack([self.colum_expancion,img_y])
        img_dy=img_dy[:-1,:]
        img_dx=img_dx[:,:-1]
        img_area=img_dx*img_dy
        img_area=cv2.bitwise_and(img_area,img_area,mask=mascara)
        #print("distancia maxima de profundidad en mm: {}".format(img_z.max()))

        area=np.sum(img_area)
        return area
    def abrir_imgs(self,path, indicador1,indicador2,nombre):
        #abrir imagenes
        img2_depth=0
        img_depth=0
        ret,img_rgb,img_depth=self.abrir_par_img(path,indicador1,indicador2,nombre)
       
        if ret:
            #quitar distorsion z
            img_depth=self.distorsion_z(img_depth,filtrar=True)
            #quitar distorsion
            img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
            # cambiar de dominio
            img2_depth=self.rgb2_depth(img_rgb)
        return ret,img2_depth,img_depth
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e

In [ ]:

from tabulate import tabulate
list_tablas=[]
list_config=[
#     "configuracion_d5_70",
            "configuracion_d5_90",
#             "configuracion_d5_110",
#             "configuracion_d5_130",
#             "configuracion_d5_150",
#             "configuracion_d5_170",
#             "configuracion_d10_120",
            "configuracion_d10_130",
#             "configuracion_d10_140",
#             "configuracion_d10_150",
#             "configuracion_d10_170",
            "configuracion_distorsion_d5_70",
            "configuracion_distorsion_d5_90",
#             "configuracion_distorsion_d5_110",
#             "configuracion_distorsion_d5_130"]
            #"configuracion_distorsion_d5_150",
            #"configuracion_distorsion_d5_170",
#             "configuracion_distorsion_d10_120
            "configuracion_distorsion_d10_140",
#             "configuracion_distorsion_d10_150",
            "configuracion_distorsion_d10_170"]
ejex=[]
ejey=[]
list_titulos=["error porcentual"]
for config in list_config:
    print(config)
    a=area("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_distorsion_90/",config)
        # abrir el par de imagenes
    list_medidas=[]
    list_error=[]
    list_media=[]
    for i in range(1,100):
        ret,img2_depth,img_depth=a.abrir_imgs(path="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/circulos",
                                        indicador1="gray",
                                        indicador2="depth",
                                        nombre=i
                                       )
        if ret:
            #segmentar
            thr,mascara,_=segmetador(img2_depth)
            #visualizar 
            #graficar([img_depth,img2_depth,mascara],1,3,["img original profundidad","img rgb en profundidad","mascara"])

            area_calculada=a.calcular_area(img_depth,mascara)
            valor_real=((150.0**2)*np.pi)/4
            #valor_real=10000
            error=a.calcular_error(valor_real,area_calculada)
            list_medidas.append(area_calculada)
            list_error.append(error)
            list_media.append(img_depth.mean()/10)
#             print("valor medio profundidad mm: ",img_depth.mean()/10)
#             print("valor real mm²: {}".format(valor_real))
#             print("area calculada mm²:{}".format(area_calculada))
#             print("porcentaje de error:{}".format(error))
#     print(valor_real)
    tabla=zip(list_medidas,list_error,list_media)
    ejex.append(list_media)
    ejey.append(np.absolute(list_error))
    list_titulos.append(config)
    print("valor real:{}".format(valor_real))
    print(tabulate(tabla,headers=["v calculado","error","distancia media"]))
    plot([list_media],[np.absolute(list_error)],1,1,["Area círculos"],[["Profundidad","Error absoluto %"]],tipo=1)
    list_media_circulos=list_media
    circulos=list_error
plot(ejex,ejey,list_titulos=list_titulos,ejes=[["profundidad","error"]],tipo=1)

In [ ]:
class medir_areas(area):
    def __init__(self,path_configuracion,nombre_configuracion):
        area.__init__(self,path_configuracion,nombre_configuracion)

    def abrir_imagenes(self,path):
        list_nombres=[]
        list_img_depth=[]
        list_mascara=[]
        list_z_maximo=[]
       
        for i in range(0,100):
            ret,img_rgb,img_depth=self.abrir_par_img(path=path, indicador_rgb="gray",
                                            indicador_depth="depth",
                                            nombre=i
                                           )
          
            #verificar existencia
            if ret:
                #abrir img referencia 
                
                #quitar distorsión y segmentar
                img_depth,mascara=self.preprocesar_imgs(img_rgb,img_depth,self.segmentador)
              
                list_nombres.append(i)
                list_img_depth.append(img_depth)
                list_mascara.append(mascara)
                list_z_maximo.append(img_depth.max())
                
   
        return list_nombres,list_img_depth,list_mascara,list_z_maximo
    def calcular(self,list_img_depth,list_mascara,valor_real):
        list_medidas=[]
        list_error=[]
        for img_depth,mascara in zip(list_img_depth,list_mascara):
            area_calculado=self.calcular_area(img_depth,mascara)
            error=self.calcular_error(valor_real,area_calculado)
            list_medidas.append(area_calculado)
            list_error.append(error)
        return list_medidas,list_error
    def segmentador(self,img,negro=True):
        n=200
        kerner=kernel = np.ones((n,n),np.uint8)
        img_gray=cv2.cvtColor(img.copy(), cv2.COLOR_RGB2GRAY)
        #img_= cv2.medianBlur(img_gray,15)
        if negro:
            criterio= cv2.THRESH_BINARY+cv2.THRESH_OTSU
            img_gray = cv2.morphologyEx(img_gray, cv2.MORPH_BLACKHAT, kernel)#MORPH_BLACKHAT

        else:
            criterio=cv2.THRESH_BINARY+cv2.THRESH_OTSU
        thr,mascara =cv2.threshold(img_gray,0,1,criterio)
        mascara.dtype=bool## remove_small solo funciona con variables binarias
        mascara=morphology.remove_small_objects(mascara,50,connectivity=10)

        mascara.dtype=np.uint8
        img_mascara=cv2.bitwise_and(img,img,mask=mascara)
        return mascara
    
    

In [ ]:
from tabulate import tabulate
medicion=medir_areas("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion/","configuracion_d5_110")

list_nombres,list_img_depth,list_mascara,list_z_maximo=medicion.abrir_imagenes("/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/circulos/")
list_medidas,list_error=medicion.calcular(list_img_depth,list_mascara,valor_real=((150.0**2)*np.pi)/4
                                         )
tabla(zip(list_z_maximo,list_medidas,list_error))
print(tabla)


# Comparar archivos de configuración

1. la clase habre un archivo de configuracion que contiene los parametros de la camara
2. la clase le calcula el area a todas las imagnes que se encuentren el directorio indicado
3. la clase almacena en una lista el area calculada por cada imagen.
4. se espera que el nombre de las imagenes este en formato sensor_numero.png. sensor=gray para el sensor RGB, y sensor=depth para el sensor de profundidad.
5. la clase almacena  el error encontrado en cada imagen.
6. la clase se caracteriza por segmentar objetos oscuros en fondo blanco.


In [ ]:
from medicion import *
from skimage import morphology
import numpy as np
import cv2
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import graficar,plot
class calcular_areas(area):
    def __init__(self,path_configuracion,nombre_configuracion):
        area.__init__(self,path_configuracion,nombre_configuracion)
    def abrir_imagenes(self,path):
        list_nombres=[]
        list_img_depth=[]
        list_mascara=[]
        list_z_maximo=[]
        
        
        for i in range(1,100):
            ret,img2_depth,img_depth=self.abrir_imgs(path=path,
                                            indicador1="gray",
                                            indicador2="depth",
                                            nombre=i
                                           )
            if ret:
                #segmentar
                thr,mascara,_=self.segmentador(img2_depth)
                list_nombres.append(i)
                list_img_depth.append(img_depth)
                list_mascara.append(mascara)
                list_z_maximo.append(img_depth[60:-60,60:-60].mean())
                
        return list_nombres,list_img_depth,list_mascara,list_z_maximo
    def calcular(self,list_img_depth,list_mascara,valor_real):
        list_medidas=[]
        list_error=[]
        for img_depth,mascara in zip(list_img_depth,list_mascara):
            area_calculada=self.calcular_area(img_depth,mascara)
            error=self.calcular_error(valor_real,area_calculada)
            list_medidas.append(area_calculada)
            list_error.append(error)
        return list_medidas,list_error
    def segmentador(self,img):
        img_gray=cv2.cvtColor(img.copy(), cv2.COLOR_RGB2GRAY)
        #img_= cv2.medianBlur(img_gray,15)
        thr,mascara =cv2.threshold(img_gray,0,1,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        mascara.dtype=bool## remove_small solo funciona con variables binarias
        mascara=morphology.remove_small_objects(mascara,50,connectivity=1)
        mascara.dtype=np.uint8
        img_mascara=cv2.bitwise_and(img,img,mask=mascara)
        return thr,mascara,img_mascara

# prueba  para comparar archivos de configuración

In [ ]:
from medicion import *
from tabulate import tabulate
#crear objeto que contiene los datos de calibración
areas=calcular_areas(path_configuracion="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/",
                    nombre_configuracion="configuracion_distorsion_d5_110"
                   )
list_nombres,list_img_depth,list_mascara,list_z_maximo=areas.abrir_imagenes(path="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/circulos/")
list_medidas,list_error=areas.calcular(list_img_depth,list_mascara,valor_real=((150.0**2)*np.pi)/4)

for i in range(len(list_nombres)):
    #graficar datos|
    print("nombre:{}".format(list_nombres[i]))
    print("valor_maximo de profundidad 100um²: {}".format(list_z_maximo[i]))
    print("valor real mm²:{}".format(((150.0**2)*np.pi)/4))
    print("valor calculado mm²: {}".format(list_medidas[i]))
    print("error %: {}".format(list_error[i]))
    

    #graficar([list_img_depth[i],list_mascara[i]],1,2,["img original profundidad","mascara"])
tabla=zip(list_nombres,list_medidas,list_error,list_z_maximo)

#tabla=[dato for dato in tabla]
print(tabulate(tabla,headers=["nombre","valor calculado [mm²]","error %","profundidad maxima [100um²] "]))
plot(list_x=[list_z_maximo],list_y=[list_error],list_titulos=["error %"],ejes=[["valor produndidad maximo","error"]],tipo=1)

# Función comparacion  

In [ ]:
from tabulate import tabulate
def comparacion(path_configuracion,list_configuracion,path_img,valor_real):
    # crear una lista que contenga objetos calcular_areas segun el archivo de configuración
    list_area=[calcular_areas(path_configuracion,config) for config in list_configuracion]
    list_nombres,list_img_depth,list_mascara,list_z_maximo=list_area[0].abrir_imagenes(path_img)
    #crear una lista donde cada elemento tiene 5 listas, nombres, mascara,valor de profundidad maxmo,medidas encontradas, error
    resultados=[area.calcular(list_img_depth,list_mascara,valor_real) for area in list_area]
    #organizar resultados
    list_medidas=[]
    list_error=[]
    for dato in resultados:
        list_medidas.append(dato[0])
        list_error.append(dato[1])
    tablas=[]
    for i in range(len(list_configuracion)):
        print(list_configuracion[i])
        tabla=zip(list_nombres,list_medidas[i],list_error[i],list_z_maximo)
        
        print(tabulate(tabla,headers=["nombre","valor calculado [mm²]","error %","profundidad maxima [100um²] "]))
    return [np.absolute(list_error[0]),list_z_maximo]


# comparación area circulos

In [ ]:

list_configuracion=[
    "70_d10_140",
    "90_d10_140",
    "110_d10_140",
    "normal_d10_140"
]
# list_configuracion=[
#     "configuracion_d5_70",
#             "configuracion_d5_90",
#             "configuracion_d5_110",
#             "configuracion_d5_130",
# #             "configuracion_d5_150",
# #             "configuracion_d5_170",
# #             "configuracion_d10_120",
# #             "configuracion_d10_130",
# #             "configuracion_d10_140",
# #             "configuracion_d10_150",
# #             "configuracion_d10_170",
#             "configuracion_distorsion_d5_70",
#             "configuracion_distorsion_d5_90",
#             "configuracion_distorsion_d5_110",
#             "configuracion_distorsion_d5_130"]
# #             #"configuracion_distorsion_d5_150",
# #             #"configuracion_distorsion_d5_170",
# #             "configuracion_distorsion_d10_120",
# #             "configuracion_distorsion_d10_130",
# #             "configuracion_distorsion_d10_140",
# #             "configuracion_distorsion_d10_150",
# #             "configuracion_distorsion_d10_170"]
ejex=[]
ejey=[]
list_titulos=["error area"]
path_carpeta="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/distorsion/"
for config in list_configuracion:
    print(path_carpeta)
    datos=comparacion(path_configuracion=path_carpeta,
            list_configuracion=[config],
            path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/circulos/",
            valor_real=((150.0**2)*np.pi)/4
           )
    ejex.append(datos[1])
    ejey.append(datos[0])
    list_titulos.append(config)
plot(ejex,ejey,list_titulos=list_titulos,ejes=[["profundidad","error"]],tipo=1)
    



In [ ]:

list_configuracion=[
#     "configuracion_d5_70",
#             "configuracion_d5_90",
#             "configuracion_d5_110",
#             "configuracion_d5_130",
#             "configuracion_d5_150",
#             "configuracion_d5_170",
            "configuracion_d10_120",
            "configuracion_d10_130",
            "configuracion_d10_140",
            "configuracion_d10_150",
            "configuracion_d10_170",
#             "configuracion_distorsion_d5_70",
#             "configuracion_distorsion_d5_90",
#             "configuracion_distorsion_d5_110",
#             "configuracion_distorsion_d5_130"]
            #"configuracion_distorsion_d5_150",
            #"configuracion_distorsion_d5_170",
            "configuracion_distorsion_d10_120",
            "configuracion_distorsion_d10_130",
            "configuracion_distorsion_d10_140",
            "configuracion_distorsion_d10_150",
            "configuracion_distorsion_d10_170"]
ejex=[]
ejey=[]
list_titulos=["error area"]
path_carpeta="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_normal/"
for config in list_configuracion:
    print(path_carpeta)
    datos=comparacion(path_configuracion=path_carpeta,
            list_configuracion=[config],
            path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/circulos/",
            valor_real=((150.0**2)*np.pi)/4
           )
    ejex.append(datos[1])
    ejey.append(datos[0])
    list_titulos.append(config)
plot(ejex,ejey,list_titulos=list_titulos,ejes=[["profundidad","error"]],tipo=1)
    




In [ ]:

list_configuracion=[
#     "configuracion_d5_70",
            "configuracion_d5_90",
#             "configuracion_d5_110",
#             "configuracion_d5_130",
#             "configuracion_d5_150",
#             "configuracion_d5_170",
#             "configuracion_d10_120",
#             "configuracion_d10_130",
             "configuracion_d10_140",
#             "configuracion_d10_150",
#             "configuracion_d10_170",
            "configuracion_distorsion_d5_70",
            "configuracion_distorsion_d5_90"]
#             "configuracion_distorsion_d5_110",
#             "configuracion_distorsion_d5_130"]
            #"configuracion_distorsion_d5_150",
            #"configuracion_distorsion_d5_170",
#             "configuracion_distorsion_d10_120
#             "configuracion_distorsion_d10_140"
#             "configuracion_distorsion_d10_150",
#             "configuracion_distorsion_d10_170"]
ejex=[]
ejey=[]
list_titulos=["error area"]
path_carpeta="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_normal/"
for config in list_configuracion:
    print(path_carpeta)
    datos=comparacion(path_configuracion=path_carpeta,
            list_configuracion=[config],
            path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/circulos/",
            valor_real=((150.0**2)*np.pi)/4
           )
    ejex.append(datos[1])
    ejey.append(datos[0])
    list_titulos.append(config)
plot(ejex,ejey,list_titulos=list_titulos,ejes=[["profundidad","error"]],tipo=1)
    

# comparción poligono

In [ ]:
#ralizar comparacion
comparacion(path_configuracion="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion/",
            list_configuracion=["configuracion_union",
                                "configuracion_d5_50",
                                "configuracion_d5_110",
                                "configuracion_d10_110"
                                ],
            path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/poligono/",
            valor_real=14614.18
           )

# comparación piñon

In [ ]:
#ralizar comparacion
comparacion(path_configuracion="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion/",
            list_configuracion=["configuracion_union",
                                "configuracion_d5_50",
                                "configuracion_d5_110",
                                "configuracion_d10_110"
                                ],
            path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/piñon/",
            valor_real=15524.847
           )

# Calcular volumen
1. solo podra calcular volumen de formamas limitadas. Esto es debido a que solo se tiene la vista superior del objeto para poder calcularla.
2. dado un objeto, denotaremos como cara superior, la superficie que mira directamente la camara. se denotara como cara inferior la superficie atras del objeto, opuesta a la carmara.
3. el sismeta medira con mayor presicion toda aquella inforamcion que este dentro un marco a 60 px por cada lado.
4. la parte inferior del objeto debe estar limitada por una superficie referencia, dicha información de la supeficie debe ser tomada o recontruida antes de poder empezar a calcular volumen,el sistema medirá con mayor presición si la superficie y la camara forman paralelas
5. si denotamos como la profundidad la distancia entre la superficie referencia y la camara, y dividimos al objeto en areas transversales, perpendiculares a z, el sistema solo podra calcular el volumen a objetos cuyas areas de la cara superior sean menores o iguales a las de la cara inferior.
![title](volumen.png)
6. en la anterior figura se mutra de forma grafica como se cacula el volumen. como se puede ver la  cara superior puede tener cualquier nivel o curva,siempre y cuando no sea mayor a la cara inferior. tambien cabe destacar como la parte inferior esta cortada por un plano.
# clase
1. la clase 

# obtner imagenes para volumen


In [ ]:
from ventana import ventana
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from scipy import stats
import cv2 as cv
import os
from openni import openni2
from openni import _openni2 as c_api
import sys
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import *
def obtener_img_calibracion(path,nueva_carpeta,n_xpaso):
    #crear objeto para verificar inclinacion
    #numero 
    paralelas=verificar_paralelas(recorte_px=60,rango_px=10,umbral=20,ancho_linea=20)
    # initialize interface
    interface=ventana("presionar f: para guardar img; c para salir")
    # comprobar carpeta de almacenamiento
    #path='/home/estufab/Escritorio/flujo de bagazo/codigo/img_calibracion'
    if  not(os.path.isdir("{}/{}/".format(path,nueva_carpeta))):
        os.mkdir("{}/{}".format(path,nueva_carpeta))
    #actualizar path
    path+=nueva_carpeta
    # Initialize the depth device
    openni2.initialize()
    dev = openni2.Device.open_any()

    # Start the depth stream
    depth_stream = dev.create_depth_stream()
    depth_stream.start()
    depth_stream.set_video_mode(c_api.OniVideoMode(pixelFormat = c_api.OniPixelFormat.ONI_PIXEL_FORMAT_DEPTH_100_UM, resolutionX = 640, resolutionY = 480, fps = 30))
    rgb_stream = dev.create_color_stream()
    rgb_stream.set_video_mode(c_api.OniVideoMode(pixelFormat=c_api.OniPixelFormat.ONI_PIXEL_FORMAT_RGB888, resolutionX= 640, resolutionY = 480, fps = 30))
    rgb_stream.start()

    def get_gray():                                                                  
        bgr   = np.frombuffer(rgb_stream.read_frame().get_buffer_as_uint16(),dtype=np.uint8).reshape(480,640,3)
        gray   = cv2.cvtColor(bgr,cv2.COLOR_BGR2RGB)
        return gray
    #contador de imagenes
    x=0
    cnt=0
    while True:
        #contador de pasos
        n=int(x/n_xpaso)
        # Grab a new depth frame


        img_gray = get_gray()    
        depth = np.frombuffer(depth_stream.read_frame().get_buffer_as_uint16(),dtype=np.uint16)
        depth.shape = (1, 480, 640)
        depth = np.concatenate((depth, depth, depth), axis=0)
        depth = np.swapaxes(depth, 0, 2)  #las columnas las vuelvo filas
        depth = np.swapaxes(depth, 0, 1) # las filas las vuelvo columnas
        depth_mod=depth.copy()
        depth_mod[np.where(depth==0)]=depth.mean()
        #pseudo color
        depth_copy=cv2.cvtColor(depth_mod, cv2.COLOR_BGR2GRAY)/256
        depth_color = cv2.applyColorMap(depth_copy.astype(np.uint8), cv2.COLORMAP_JET)#COLORMAP_JET##COLORMAP_PINK
        #verificar inclinacion
        img_paralelas,ex,ey=paralelas.verificar(depth_mod.copy(),depth_color.copy())
        ex=int(ex)
        ey=int(ey)
        desviacion=int(depth_mod.std())
        media=int(depth_mod.mean())
        #visualizacion
        interface.visualizar([img_gray,depth_color,img_paralelas],img_gray.shape[0:2],2,2,["RGB","DEPTH: {}||{}".format(media,4000+(n)*800+1520-600)," x:{}  |  y:{} | d:{}".format(ex,ey,desviacion)])
                                                                                             


        key = cv2.waitKey(1) & 0xFF
        # If the 'c' key is pressed, break the while loopc
        if key == ord("c"):
                       break
        if key == ord("f"):
            x+=1
            path_gray=os.path.join(path, 'gray_{}.png'.format(x))
            path_matrix_depth=os.path.join(path, 'gray_{}.png'.format(x))
            path_depth_16=os.path.join(path, 'depth_{}.png'.format(x))
            cv2.imwrite(path_depth_16,depth)
            np.save(path_matrix_depth,depth)
            ret=cv2.imwrite(path_gray,img_gray)
            print(x,depth_mod.mean())
            print ("end-start")
        if key == ord("g"):
            cnt+=1
            
            
            path_ref=os.path.join(path, 'ref_{}.png'.format(x))
            cv2.imwrite(path_ref,depth)
      
            print("referencia:",cnt,depth_mod.mean(),x)
            print ("end-start")

    openni2.unload()
    cv2.destroyAllWindows()

In [ ]:
path='/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/'

nueva_carpeta="volumen_cubo3"
a=obtener_img_calibracion(path,nueva_carpeta,n_xpaso=4)

# segmentador objetos blancos fondo oscuro

In [ ]:
from skimage import morphology
import numpy as np
import cv2
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import graficar
def segmentador(img,negro=True):
    n=200
    kerner=kernel = np.ones((n,n),np.uint8)
    img_gray=cv2.cvtColor(img.copy(), cv2.COLOR_BGR2YUV)
    img_gray[:,:,0]=cv2.equalizeHist(img_gray[:,:,0])
    img_gray=cv2.cvtColor(img.copy(), cv2.COLOR_YUV2BGR)
    img_gray=cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)
    
    #img_= cv2.medianBlur(img_gray,15)
    if negro:
        umbral=50
        img_gray[:,:50]=255
        img_gray[img_gray<=umbral]=1
        img_gray[img_gray>umbral]=0
    else:
        umbral=200
        img_gray[:,:50]=0
        img_gray[img_gray<=umbral]=0
        img_gray[img_gray>umbral]=1
       
   
    mascara=img_gray
    mascara.dtype=bool## remove_small solo funciona con variables binarias
    mascara=morphology.remove_small_objects(mascara,50,connectivity=10)
    mascara.dtype=np.uint8
    img_mascara=cv2.bitwise_and(img,img,mask=mascara)

    return umbral,mascara,img_mascara
for i in range(1,80):
    # abrir img
    print (i)
    img=cv2.imread("/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/cubo2/gray_{}.png".format(i))
    
    thr,mascara,img_mascara=segmentador(img)
    print("humbral: {}".format(thr))

    graficar([img,mascara,img_mascara],1,3,["original","mascara","original X mascara"])

In [ ]:
# clase

In [ ]:
import numpy as np
from medicion import *
import matplotlib.pyplot as plt
class volumen(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion)
        self.fx=self.depth_intrinseca[0,0]
        self.fy=self.depth_intrinseca[1,1]
        self.constante_area=1/(self.fx*self.fy)
       
 
    def preprocesar_imgs(self,img_rgb,img_depth,f_segmentador,negro=True):
        #quitar distorsion z
        img_depth=self.distorsion_z(img_depth,filtrar=True)
     
        #quitar distorsion uv
        img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        # cambiar de dominio rgb a depth
        img2_depth=self.rgb2_depth(img_rgb)
        # encontrar mascara
        ## hay que corregir 
        umbral,mascara,_=f_segmentador(img2_depth,negro)
        return img_depth,mascara
    def calcular_volumen(self,img,mascara,img_ref):
        print(img_ref.mean())
        img_z=cv2.bitwise_and(img,img,mask=mascara)
        img_ref=cv2.bitwise_and(img_ref,img_ref,mask=mascara)
        
        img_z=np.array(img_z/10,dtype=np.float64)
        graficar([img_z],1,1,["img x mascara"])
        objeto=img_z.copy()[img_z>0].reshape(-1)
        ref=img_ref.copy()[img_ref>0].reshape(-1)
        i_objeto=range(objeto.size)
        i_ref=range(ref.size)
        #plot(list_x=[i_objeto,i_ref],list_y=[objeto,ref],filas=1,colum=2,list_titulos=["altura objeto","altura referencia"],ejes=[["px index","profundidad"],["px index","profundidad"]])
        img_z2=img_z.copy()*img_z.copy()
        
        img_area=self.constante_area*img_z2
        img_altura=img_ref.copy()-img_z.copy()
        
        img_volumen=img_area*img_altura
        volumen=np.sum(img_volumen)
        return volumen
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def mejorar_referencia(self,img_ref):

        img_ref[img_ref==0]=img_ref.mean()
        img_ref=self.distorsion_z(img_ref,1)
        return img_ref
    def abrir_img_ref(self,path,nombre):
        path_ref=os.path.join(path, 'ref_{}.png'.format(nombre))
        ret=cv2.haveImageReader(path_ref)
        img_ref=0
        if ret:
            img_ref=cv2.imread(path_ref,cv2.IMREAD_UNCHANGED)[:,:,0]
            img_ref=self.mejorar_referencia(img_ref)
        return ret,img_ref/10 

# calcular volumen a un paquete de imagnes

In [ ]:
from medicion import *
from skimage import morphology
import numpy as np
import cv2
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import graficar, plot
class calcular_volumenes(volumen):
    def __init__(self,path_configuracion,nombre_configuracion):
        volumen.__init__(self,path_configuracion,nombre_configuracion)
    def abrir_imagenes(self,path,negro):
        list_nombres=[]
        list_img_depth=[]
        list_mascara=[]
        list_z_maximo=[]
        list_ref=[]
        _,img_ref=self.abrir_img_ref(path,0)
        #img_ref=imshow("{}ref_0.png".format(path),IMREAD_UNCHANGED)
        for i in range(0,70):
          
            ret,img_rgb,img_depth=self.abrir_par_img(path=path, indicador_rgb="gray",
                                            indicador_depth="depth",
                                            nombre=i)
          
                
            #verificar existencia
            if ret:
                #abrir img referencia 
                if (i-1)%4==0:
                    _,img_ref=self.abrir_img_ref(path,"ref_{}".format(i-1))
                #quitar distorsión y segmentar
                img_depth,mascara=self.preprocesar_imgs(img_rgb,img_depth,self.segmentador)
              
                list_nombres.append(i)
                list_img_depth.append(img_depth/10)
                list_mascara.append(mascara)
                list_z_maximo.append(img_depth.max())
                list_ref.append(img_ref/10)
            
        return list_nombres,list_img_depth,list_mascara,list_z_maximo,list_ref
    def calcular(self,list_img_depth,list_mascara,list_ref,valor_real):
        list_medidas=[]
        list_error=[]
        for img_depth,mascara,img_ref in zip(list_img_depth,list_mascara,list_ref):
            volumen_calculado=self.calcular_volumen(img_depth,mascara,img_ref)
            error=self.calcular_error(valor_real,volumen_calculado)
            list_medidas.append(volumen_calculado)
            list_error.append(error)
        return list_medidas,list_error
    def segmentador(self,img,negro=True):
        n=200
        kerner=kernel = np.ones((n,n),np.uint8)
        img_gray=cv2.cvtColor(img.copy(), cv2.COLOR_BGR2YUV)
        img_gray[:,:,0]=cv2.equalizeHist(img_gray[:,:,0])
        img_gray=cv2.cvtColor(img.copy(), cv2.COLOR_YUV2BGR)
        img_gray=cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)

        #img_= cv2.medianBlur(img_gray,15)
        if negro:
            umbral=50
            img_gray[:,:50]=255
            img_gray[img_gray<=umbral]=1
            img_gray[img_gray>umbral]=0
        else:
            umbral=200
            img_gray[:,:50]=0
            img_gray[img_gray<=umbral]=0
            img_gray[img_gray>umbral]=1


        mascara=img_gray
        mascara.dtype=bool## remove_small solo funciona con variables binarias
        mascara=morphology.remove_small_objects(mascara,50,connectivity=10)
        mascara.dtype=np.uint8
        img_mascara=cv2.bitwise_and(img,img,mask=mascara)

        return mascara,img_mascara

# comparador de archivos de configuración 

In [ ]:
from tabulate import tabulate
def comparacion(path_configuracion,list_configuracion,path_img,valor_real):
    # crear una lista que contenga objetos calcular_areas segun el archivo de configuración
    list_volumen=[calcular_volumenes(path_configuracion,config) for config in list_configuracion]
    list_nombres,list_img_depth,list_mascara,list_z_maximo,list_ref=list_volumen[0].abrir_imagenes(path_img,negro=True)
    #crear una lista donde cada elemento tiene 5 listas, nombres, mascara,valor de profundidad maxmo,medidas encontradas, error
    resultados=[volumen.calcular(list_img_depth,list_mascara,list_ref,valor_real) for volumen in list_volumen]
    #organizar resultados
    list_medidas=[]
    list_error=[]
    for dato in resultados:
        list_medidas.append(dato[0])
        list_error.append(dato[1])
    tablas=[]
    for i in range(len(list_configuracion)):
        print(list_configuracion[i])
        tabla=zip(list_nombres,list_medidas[i],list_error[i],list_z_maximo)
        print(tabulate(tabla,headers=["nombre","valor calculado [mm²]","error %","profundidad maxima [100um²] "]))
    return [list_error[i],list_z_maximo]
#ralizar comparacion
list_configuracion=[
#     "configuracion_d5_70",
            "configuracion_d5_90 (d5)",
#             "configuracion_d5_110",
#             "configuracion_d5_130",
#             "configuracion_d5_150",
#             "configuracion_d5_170",
#             "configuracion_d10_120",
#             "configuracion_d10_130",
            "configuracion_d10_140",
#             "configuracion_d10_150",
#             "configuracion_d10_170",
            "configuracion_distorsion_d5_70",
            "configuracion_distorsion_d5_90"]
#             "configuracion_distorsion_d5_110",
#             "configuracion_distorsion_d5_130"]
            #"configuracion_distorsion_d5_150",
            #"configuracion_distorsion_d5_170",
#             "configuracion_distorsion_d10_120
#             "configuracion_distorsion_d10_140"]
#             "configuracion_distorsion_d10_150",
#             "configuracion_distorsion_d10_170"]
ejex=[]
ejey=[]
list_titulos=["grafica error porcentual"]
for config in list_configuracion:
    tabla=comparacion(path_configuracion="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_120/",
                list_configuracion=[config
                                    ],
                path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/cubo2/",
                valor_real=1000100
               )
    plot([tabla[1]],[tabla[0]],list_titulos=["Error"],ejes=[["profundidad","error"]],tipo=1)
    list_titulos.append(config)
    ejex.append(tabla[1])
    ejey.append(tabla[0])
plot(ejex,ejey,list_titulos=list_titulos,ejes=[["profundidad","error"]],tipo=1)

In [ ]:
from tabulate import tabulate
def comparacion(path_configuracion,list_configuracion,path_img,valor_real):
    # crear una lista que contenga objetos calcular_areas segun el archivo de configuración
    list_volumen=[calcular_volumenes(path_configuracion,config) for config in list_configuracion]
    list_nombres,list_img_depth,list_mascara,list_z_maximo,list_ref=list_volumen[0].abrir_imagenes(path_img,negro=True)
    #crear una lista donde cada elemento tiene 5 listas, nombres, mascara,valor de profundidad maxmo,medidas encontradas, error
    resultados=[volumen.calcular(list_img_depth,list_mascara,list_ref,valor_real) for volumen in list_volumen]
    #organizar resultados
    list_medidas=[]
    list_error=[]
    for dato in resultados:
        list_medidas.append(dato[0])
        list_error.append(dato[1])
    tablas=[]
    for i in range(len(list_configuracion)):
        print(list_configuracion[i])
        tabla=zip(list_nombres,list_medidas[i],list_error[i],list_z_maximo)
        print(tabulate(tabla,headers=["nombre","valor calculado [mm²]","error %","profundidad maxima [100um²] "]))
    return [np.absolute(list_error[i]),list_z_maximo]
#ralizar comparacion
import numpy as np
from medicion import *
import matplotlib.pyplot as plt
class volumen(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion)
        self.fx=self.depth_intrinseca[0,0]
        self.fy=self.depth_intrinseca[1,1]
        self.constante_area=1/(self.fx*self.fy)
       
 
    def preprocesar_imgs(self,img_rgb,img_depth,f_segmentador,negro=True):
        #quitar distorsion z
        img_depth=self.distorsion_z(img_depth,filtrar=True)
     
        #quitar distorsion uv
        img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        # cambiar de dominio rgb a depth
        img2_depth=self.rgb2_depth(img_rgb)
        # encontrar mascara
        umbral,mascara,_=f_segmentador(img2_depth,negro)
        return img_depth,mascara
    def calcular_volumen(self,img,mascara,img_ref):
        print(img_ref.mean())
        img_z=cv2.bitwise_and(img,img,mask=mascara)
        img_ref=cv2.bitwise_and(img_ref,img_ref,mask=mascara)
        
        img_z=np.array(img_z/10,dtype=np.float64)
        graficar([img_z],1,1,["img x mascara"])
        objeto=img_z.copy()[img_z>0].reshape(-1)
        ref=img_ref.copy()[img_ref>0].reshape(-1)
        i_objeto=range(objeto.size)
        i_ref=range(ref.size)
        #plot(list_x=[i_objeto,i_ref],list_y=[objeto,ref],filas=1,colum=2,list_titulos=["altura objeto","altura referencia"],ejes=[["px index","profundidad"],["px index","profundidad"]])
        img_z2=img_z.copy()*img_z.copy()
        
        img_area=self.constante_area*img_z2
        img_altura=img_ref.copy()-img_z.copy()
        
        img_volumen=img_area*img_altura
        volumen=np.sum(img_volumen)
        return volumen
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def mejorar_referencia(self,img_ref):

        img_ref[img_ref==0]=img_ref.mean()
        img_ref=self.distorsion_z(img_ref,1)
        return img_ref
    def abrir_img_ref(self,path,nombre):
        path_ref=os.path.join(path, 'ref_{}.png'.format(nombre))
        ret=cv2.haveImageReader(path_ref)
        img_ref=0
        if ret:
            img_ref=cv2.imread(path_ref,cv2.IMREAD_UNCHANGED)[:,:,0]
            img_ref=self.mejorar_referencia(img_ref)
        return ret,img_ref/10 
ejex=[]
ejey=[]
list_titulos=["grafica error porcentual"]
for config in list_configuracion:
    tabla=comparacion(path_configuracion="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/",
                list_configuracion=[config
                                    ],
                path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/cubo2/",
                valor_real=1000100
               )
    plot([tabla[1]],[tabla[0]],list_titulos=["Error"],ejes=[["profundidad","error"]],tipo=1)
    list_titulos.append(config)
    ejex.append(tabla[1])
    ejey.append(tabla[0])
plot(ejex,ejey,list_titulos=list_titulos,ejes=[["profundidad","error"]],tipo=1)

In [ ]:
comparacion(path_configuracion="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_/",
                list_configuracion=[#"configuracion_distorsion_d5_70",
            #"configuracion_d5_70",
            "configuracion_d5_90",
#             "configuracion_d5_110",
#             "configuracion_d5_130",
#             "configuracion_d5_150",
#             "configuracion_d5_170",
            "configuracion_d10_120",
            "configuracion_d10_130",
            "configuracion_d10_140",
            "configuracion_d10_150",
            "configuracion_d10_170",
#             "configuracion_distorsion_d5_70",
#             "configuracion_distorsion_d5_90",
#             "configuracion_distorsion_d5_110",
#             "configuracion_distorsion_d5_130"
            #"configuracion_distorsion_d5_150",
            #"configuracion_distorsion_d5_170",
            "configuracion_distorsion_d10_120",
            "configuracion_distorsion_d10_130",
            "configuracion_distorsion_d10_140",
            "configuracion_distorsion_d10_150",
            "configuracion_distorsion_d10_170"
                                    ],
                path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_medicion/volumen/cubo2/",
                valor_real=1000010
               )

# medición teniendo en cuenta superficies y no planos


In [ ]:

    
import numpy as np
from medicion import *
import matplotlib.pyplot as plt
class volumen(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion)
        self.fx=self.depth_intrinseca[0,0]
        self.fy=self.depth_intrinseca[1,1]
        self.cx=self.depth_intrinseca[0,2]
        self.cy=self.depth_intrinseca[0,2]
        self.xn=0
        self.yn=0
        self.fila_expancion=0
        self.colum_expancion=0
       
 
    def preprocesar_imgs(self,img_rgb,img_depth,f_segmentador,negro=True):
        #quitar distorsion z
        img_depth=self.distorsion_z(img_depth,filtrar=True)
     
        #quitar distorsion uv
        img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        # cambiar de dominio rgb a depth
        img2_depth=self.rgb2_depth(img_rgb)
        # encontrar mascara
        umbral,mascara,_=f_segmentador(img2_depth,negro)
        return img_depth,mascara
    def calcular_volumen(self,img,mascara,img_ref):
        if (isinstance(self.yn*self.xn,int)):
            v,u=np.mgrid[0:img.shape[0],0:img.shape[1]]
            self.yn=(v-self.cy)/self.fy
            self.xn=(u-self.cx)/self.fx
            self.fila_expancion=np.zeros([img.shape[0],1])
            self.colum_expancion=np.zeros([1,img.shape[1]])
        print(img_ref.mean())
        #img_z=cv2.bitwise_and(img,img,mask=mascara)
        img_ref=cv2.bitwise_and(img_ref,img_ref,mask=mascara)
        img_z=np.array(img/10,dtype=np.float64)
        img_x=self.xn*img_z
        img_y=self.yn*img_z
#       img_z=cv2.bitwise_and(img,img,mask=mascara)
       
        # generar distancias en x y en y
       
        img_dx=np.hstack([self.fila_expancion,img_x])-np.hstack([img_x,self.fila_expancion])
        img_dx=np.hstack([img_x,self.fila_expancion])-np.hstack([self.fila_expancion,img_x,])
       
        img_dy=np.vstack([img_y,self.colum_expancion])-np.vstack([self.colum_expancion,img_y])
        img_dy=img_dy[:-1,:]
        img_dx=img_dx[:,:-1]
        img_area=img_dx*img_dy
        img_area=cv2.bitwise_and(img_area,img_area,mask=mascara)
        
        graficar([img_z],1,1,["img x mascara"])
        objeto=img_z.copy()[img_z>0].reshape(-1)
        ref=img_ref.copy()[img_ref>0].reshape(-1)
        i_objeto=range(objeto.size)
        i_ref=range(ref.size)
        #plot(list_x=[i_objeto,i_ref],list_y=[objeto,ref],filas=1,colum=2,list_titulos=["altura objeto","altura referencia"],ejes=[["px index","profundidad"],["px index","profundidad"]])
        img_z2=img_z.copy()*img_z.copy()
        
    
        img_altura=img_ref.copy()-img_z.copy()
        
        img_volumen=img_area*img_altura
        volumen=np.sum(img_volumen)
        return volumen
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def mejorar_referencia(self,img_ref):

        img_ref[img_ref==0]=img_ref.mean()
        img_ref=self.distorsion_z(img_ref,1)
        return img_ref
    def abrir_img_ref(self,path,nombre):
        path_ref=os.path.join(path, 'ref_{}.png'.format(nombre))
        ret=cv2.haveImageReader(path_ref)
        img_ref=0
        if ret:
            img_ref=cv2.imread(path_ref,cv2.IMREAD_UNCHANGED)[:,:,0]
            img_ref=self.mejorar_referencia(img_ref)
        return ret,img_ref/10 

# medidor de volumen final, para unir con el segmentador

# clase
1. el constructur de la clase resive el directorio y el nombre del archivo de configuración para cargar todos los datos de calibración
2. tiene una función preprocesar_imgs la cual se encarga de resivir la imagen RGB, la imagen de Depth y una función para segmentar. esta función quita la distorsión a ambas imagenes lleva la información RGB al dominio Depth y la segmetan con ayuda de la función entregada.
3. calcular_volumen: resive  resive la imagen Depth, la imagen mascara que contine la segementación del objeto y una imagen de referencia depth la cual es el fondo sin objeto alguno,
4. la clase tiene funciones adicionales para abrir la imagen de referencia, mejorarla, asi como tambien una función para calcular el error absoluto porcentual de una medida.
5. la imagen de referencia puede encontrarse en una carpeta diferente, solo hay que especificar el directorio. cuando se entrega la imagen al ala función abrir_ref. solo hay que entregarle el nombre

In [ ]:
import numpy as np
from medicion import *
import matplotlib.pyplot as plt
class volumen(imagen):
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion)
        self.fx=self.depth_intrinseca[0,0]
        self.fy=self.depth_intrinseca[1,1]
        self.constante_area=1/(self.fx*self.fy)
       
 
    def preprocesar_imgs(self,img_rgb,img_depth,f_segmentador):
        #quitar distorsion z
        img_depth=self.distorsion_z(img_depth,filtrar=True)
     
        #quitar distorsion uv
        img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        # cambiar de dominio rgb a depth
        img2_depth=self.rgb2_depth(img_rgb)
        # encontrar mascara
        ## hay que corregir 
        mascara=f_segmentador(img2_depth)
        return img_depth,mascara
    def calcular_volumen(self,img,mascara,img_ref):
        print(img_ref.mean())
        img_z=cv2.bitwise_and(img,img,mask=mascara)
        img_ref=cv2.bitwise_and(img_ref,img_ref,mask=mascara)
        
        img_z=np.array(img_z/10,dtype=np.float64)
        graficar([img_z],1,1,["img x mascara"])
        objeto=img_z.copy()[img_z>0].reshape(-1)
        ref=img_ref.copy()[img_ref>0].reshape(-1)
        i_objeto=range(objeto.size)
        i_ref=range(ref.size)
        #plot(list_x=[i_objeto,i_ref],list_y=[objeto,ref],filas=1,colum=2,list_titulos=["altura objeto","altura referencia"],ejes=[["px index","profundidad"],["px index","profundidad"]])
        img_z2=img_z.copy()*img_z.copy()
        
        img_area=self.constante_area*img_z2
        img_altura=img_ref.copy()-img_z.copy()
        
        img_volumen=img_area*img_altura
        volumen=np.sum(img_volumen)
        return volumen
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def mejorar_referencia(self,img_ref):

        img_ref[img_ref==0]=img_ref.mean()
        img_ref=self.distorsion_z(img_ref,1)
        return img_ref
    def abrir_img_ref(self,path,nombre):
        path_ref=os.path.join(path, '{}.png'.format(nombre))
        ret=cv2.haveImageReader(path_ref)
        img_ref=0
        if ret:
            img_ref=cv2.imread(path_ref,cv2.IMREAD_UNCHANGED)[:,:,0]
            img_ref=self.mejorar_referencia(img_ref)
        return ret,img_ref/10 

In [ ]:
from medicion import volumen
volumenes=volumen("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_120/","configuracion_d5_90")

In [ ]:
    path_ref="/home/estufab4/Desktop/flujo de bagazo/codigo/bagazo/bagazosinfondo1_1/"
    nombre_img_ref="depth_1"
    img=volumenes.abrir_img_ref(path_ref,nombre_img_ref)
    print(img)